***
<p style="font-size:26px;">
<font color=red> Deploy and Invoke your model through Data Science Model Deployment</font>
<p style="margin-left:10%; margin-right:10%;">by the <font color=teal> Oracle Cloud Infrastructure</font></p>

***

## Overview:
Load pre-trained model for and deploy it.


We demonstrate:
1. Load saved model from previous notebook
2. prepare and deploy the model using ADS

---

## Contents:

* <a href='#data'>Load data</a>
* <a href='#loadModel'>Load Model</a>
* <a href='#serialize'>Serialization and model deploymnet </a>
* <a href='#invoke_endpoint'>Invoke endpoint</a>
* <a href='#cleanup'>Clean up and delete deployed model</a>
---

### Load basic packages

In [1]:
path = '/home/datascience/RedBull-Racining-TimeToPit/notebooks'
data_path = '../../RedBull-Racining-TimeToPit/data/'

In [2]:
import os
os.chdir(path)
import pandas as pd
import logging
import json
import pickle
import requests
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

<a id='data'></a>
# Load data 

1. Load data `final_data.csv`
2. Filter out races with lesser than 5 or higher than 50 laps
2. Bucketize the target variable (StintLen)

In [3]:
df = pd.read_csv(data_path+'final_data.csv',)

## remove any StintLen lesser than 5 and greather than 35
## SintNumber==1
df = df[(df['StintLen']>5) & (df['StintLen']<=35) & (df['Stint']==1)] #& (df['Stint']==1)


## bucketize `target variable` and assign a label to each bucket
bins = np.array([5,10,15,20,25,30,35])
labels = np.arange(len(bins)-1)

df['classLabels'] = pd.cut(df.StintLen, bins=bins,labels=labels)

print('-'*80,'\n', 'Size of Data: ', df.shape,'\n', '-'*80,'\n')

-------------------------------------------------------------------------------- 
 Size of Data:  (1245, 29) 
 -------------------------------------------------------------------------------- 



In [4]:
x = df[['EventName','Compound','Driver', 'TyreAge', 
           'meanAirTemp', 'meanTrackTemp', 'meanHumid', 'Rainfall', 
           'GridPosition', 'Position','CircuitLength','designedLaps','classLabels'] ].reset_index(drop=True)
categorical_cols = []
numerical_cols = []


y = x[0:10].classLabels #pd.Categorical(x.classLabels).codes
x = x[0:10].drop(['classLabels'],axis=1)

# X_train_xgbc, X_test_xgbc, y_train_xgbc, y_test_xgbc = train_test_split(x, y, test_size=0.25, random_state=42)
# print('Regression data size for train and test: ', len(X_train_xgbc), len(X_test_xgbc),
#       '\nClassification data size for train and test: ', len(X_train_xgbc), len(y_test_xgbc))

<a id='loadModel'></a>
# Load Model

In [13]:
import joblib
ppl_model = joblib.load(data_path+'gbm_model.pkl')

<a id='serialize'></a>
# Serialization and Model Deployment

The Sklearn framework makes it easy to deploy a scikit-learn model into production. The `SklearnModel()` constructor takes a scikit-learn model and converts it into an `SklearnModel` object. To deploy the model into production, you need to prepare the model artifact, verify that the artifact works, save the model to the model catalog, and then deploy it.

ADS provides a number of methods that greatly simplify the model deployment process. It also provides the `.summary_status()` method that outputs a dataframe that defines the steps, status, and detailed information about each step. 

<a id='serialize_sklearnmodel'></a>
## Create a SklearnModel

The `SklearnModel()` constructor takes a scikit-learn model along with the path that you want to use to store the model artifacts. An `SklearnModel` object is returned, and it is used to manage the deployment.

The next cell creates a model artifact directory. This directory is used to store the artifacts that are needed to deploy the model. It also creates the `SklearnModel` object.

In [14]:
pwd

'/home/datascience/RedBull-Racining-TimeToPit/notebooks'

In [15]:
from ads.catalog.model import ModelCatalog
from ads.common.model_metadata import UseCaseType
from ads.model.framework.sklearn_model import SklearnModel
import ads

In [16]:
ads.set_auth("resource_principal")

In [17]:
artifact_dir = f"../md-rbc-Pipeline-model"#tempfile.mkdtemp()
print(f"Model artifact director: {artifact_dir}")
sklearn_model = SklearnModel(estimator=ppl_model, artifact_dir=artifact_dir)

Model artifact director: ../md-rbc-Pipeline-model


The `.summary_status()` method of the `SklearnModel` class is a handy method to keep track of the progress that you are making in deploying the model. It creates a dataframe that lists the deployment steps, thier status, and details about them. The next cell returns the summary status dataframe. It shows that the initiate step has been completed.

In [18]:
sklearn_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Available     Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Not Available Local tested .predict from score.py                               
save()    Not Available Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Not Available Deployed the model                                                
predict() Not Available Called deployment predict endpoint

<a id='serialize_prepare'></a>
## Prepare

The prepare step is performed by the `.prepare()` method of the `SklearnModel` class. It creates a number of customized files that are used to run the model once it is deployed. These include:

* `input_schema.json`: A JSON file that defines the nature of the features of the `X_sample` data. It includes metadata such as the data type, name, constraints, summary statistics, feature type, and more.
* `model.joblib`: This is the default filename of the serialized model. It can be changed with the `model_file_name` attribute. By default, the model is stored in a joblib file. The parameter `as_onnx` can be used to save it in the ONNX format.
* `output_schema.json`: A JSON file that defines the nature of the dependent variable in the `y_sample` data. It includes metadata such as the data type, name, constraints, summary statistics, feature type, and more.
* `runtime.yaml`: This file contains information that is needed to set up the runtime environment on the deployment server. It has information about which conda environment was used to train the model, and what environment should be used to deploy the model. The file also specifies what version of Python should be used.
* `score.py`: This script contains the `load_model()` and `predict()` functions. The `load_model()` function understands the format the model file was saved in and loads it into memory. The `.predict()` method is used to make inferences in a deployed model. There are also hooks that allow you to perform operations before and after inference. You can modify this script to fit your specific needs.

To create the model artifacts, you use the `.prepare()` method. There are a number of parameters that allow you to store model provenance information. In the next cell, the `conda_env` variable defines the slug of the conda environment that was used to train the model, and also the conda environment that should be used for deployment. Note that you can only pass in slugs to `inference_conda_env` or `training_conda_env` if it's a service environment. Otherwise, you must pass in the full path of the conda envvironment along with the python version through `inference_python_version` and `training_python_version`.

In [20]:
conda_env = 'generalml_p37_cpu_v1'

sklearn_model.prepare(
    inference_conda_env=conda_env,
    training_conda_env=conda_env,
    use_case_type=UseCaseType.BINARY_CLASSIFICATION,
    X_sample=x, #X_test_xgbc,
    y_sample=y, #y_test_xgbc,
)

The next cell uses the `.summary_status()` method to show you that the prepare step finished, and what tasks were completed.

Make sure you apply the following changes to the end of score.py in case you want the endpoint to return probability of detection along with binary classification results:

`input = pre_inference(data, input_schema_path)`

`yhat_bin = post_inference(model.predict(input))`

`yhat_prob = post_inference(model.predict_proba(input))`

`return {'prediction': yhat_bin,'Probability':yhat_prob}`

In [21]:
sklearn_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Available     Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Not Available Deployed the model                                                
predict() Not Available Called deployment predict endpoint

The `.prepare()` method has created the following files. These files are fully functional. However, you can modify them to fit your specific needs.

In [22]:
os.listdir(artifact_dir)

['score.py',
 '.ipynb_checkpoints',
 'output_schema.json',
 'runtime.yaml',
 'score-Copy1.py',
 'input_schema.json',
 'model.joblib']

Once the artifacts have been created, there are a number of attributes in the `SklearnModel` object that provides metadata about the model. The `.runtime` attribute details the model deployment settings and model provenance data.

In [23]:
sklearn_model.runtime_info

model_artifact_version: '3.0'
model_deployment:
  inference_conda_env:
    inference_env_path: oci://service-conda-packs@id19sfcrra6z/service_pack/cpu/General_Machine_Learning_for_CPUs_on_Python_3.7/1.0/generalml_p37_cpu_v1
    inference_env_slug: generalml_p37_cpu_v1
    inference_env_type: data_science
    inference_python_version: '3.7'
model_provenance:
  project_ocid: ''
  tenancy_ocid: ''
  training_code:
    artifact_directory: /home/datascience/RedBull-Racining-TimeToPit/md-rbc-Pipeline-model
  training_compartment_ocid: ''
  training_conda_env:
    training_env_path: oci://service-conda-packs@id19sfcrra6z/service_pack/cpu/General_Machine_Learning_for_CPUs_on_Python_3.7/1.0/generalml_p37_cpu_v1
    training_env_slug: generalml_p37_cpu_v1
    training_env_type: data_science
    training_python_version: '3.7'
  training_region: ''
  training_resource_ocid: ''
  user_ocid: ''
  vm_image_internal_id: ''

The `.schema_input` attribute provides metadata on the features that were used to train the model. You can use this information to determine what data must be provided to make model inferences. Each feature in the model has a section that defines the dtype, feature type, name, and if it is required. The metadata also includes the summary statistics associated with the feature type.

In [24]:
sklearn_model.schema_input

schema:
- description: GridPosition
  domain:
    constraints: []
    stats:
      count: 10.0
      lower quartile: 7.5
      mean: 11.7
      median: 12.5
      sample maximum: 20.0
      sample minimum: 4.0
      skew: 0.006
      standard deviation: 5.122
      upper quartile: 14.75
    values: Continuous
  dtype: float64
  feature_type: Continuous
  name: GridPosition
  order: 8
  required: true
- description: EventName
  domain:
    constraints: []
    stats:
      count: 10
      unique: 1
    values: String
  dtype: object
  feature_type: String
  name: EventName
  order: 0
  required: true
- description: meanHumid
  domain:
    constraints: []
    stats:
      count: 10.0
      lower quartile: 51.65
      mean: 52.211
      median: 51.695
      sample maximum: 54.197
      sample minimum: 51.65
      skew: 1.611
      standard deviation: 0.924
      upper quartile: 52.353
    values: Continuous
  dtype: float64
  feature_type: Continuous
  name: meanHumid
  order: 6
  required

The `.metadata_custom` attribute provides custom metadata that contains information on the category of the metadata, description, key, and value.

In [25]:
sklearn_model.metadata_custom

data:
- category: Training Environment
  description: The URI of the training conda environment.
  key: CondaEnvironmentPath
  value: oci://service-conda-packs@id19sfcrra6z/service_pack/cpu/General_Machine_Learning_for_CPUs_on_Python_3.7/1.0/generalml_p37_cpu_v1
- category: Training Profile
  description: The model serialization format.
  key: ModelSerializationFormat
  value: joblib
- category: Training Environment
  description: The slug name of the training conda environment.
  key: SlugName
  value: generalml_p37_cpu_v1
- category: Training Environment
  description: The conda environment type, can be published or datascience.
  key: EnvironmentType
  value: data_science
- category: Other
  description: ''
  key: ClientLibrary
  value: ADS
- category: Training Environment
  description: The conda environment where the model was trained.
  key: CondaEnvironment
  value: oci://service-conda-packs@id19sfcrra6z/service_pack/cpu/General_Machine_Learning_for_CPUs_on_Python_3.7/1.0/genera

The `.metadata_provenance` contains information about the code and training data that was used to create the model. This information is most useful when a Git repository is being used to manage the code for training the model. This is considered a best practice because it allows you to do things like reproduce a model, perform forensic on the model, and so on.

In [28]:
#sklearn_model.metadata_provenance

The `.metadata_taxonomy` is a key-value store that has information about the classification or taxonomy of the model. This can include information such as the model framework, use case type, hyperparameters, and more.

In [29]:
sklearn_model.metadata_taxonomy

data:
- key: Framework
  value: scikit-learn
- key: Algorithm
  value: Pipeline
- key: UseCaseType
  value: binary_classification
- key: ArtifactTestResults
  value: null
- key: Hyperparameters
  value:
    memory: None
    model: LGBMClassifiermax_depth=5, n_estimators=50
    model__boosting_type: gbdt
    model__class_weight: None
    model__colsample_bytree: '1.0'
    model__importance_type: split
    model__learning_rate: '0.1'
    model__max_depth: '5'
    model__min_child_samples: '20'
    model__min_child_weight: '0.001'
    model__min_split_gain: '0.0'
    model__n_estimators: '50'
    model__n_jobs: '-1'
    model__num_leaves: '31'
    model__objective: None
    model__random_state: None
    model__reg_alpha: '0.0'
    model__reg_lambda: '0.0'
    model__silent: warn
    model__subsample: '1.0'
    model__subsample_for_bin: '200000'
    model__subsample_freq: '0'
    preprocessor: "ColumnTransformertransformers=['cat',\n                                 Pipelinesteps=['encoder'

<a id='serialize_verify'></a>
## Verify

If you modify the `score.py` file that is part of the model artifacts, then you should verify it. The verify step allows you to test those changes without having to deploy the model. This allows you to debug your code without having to save the model to the model catalog and then deploy it. The `.verify()` method takes a set of test parameters and performs the prediction by calling the `predict` function in `score.py`. It also runs the `load_model` function.

The next cell simulates a call to a deployed model without having to actually deploy the model. It passes in test values and returns the predictions.

In [30]:
print(y[0:3])

0    4
1    0
2    0
Name: classLabels, dtype: category
Categories (6, int64): [0 < 1 < 2 < 3 < 4 < 5]


In [31]:
sklearn_model.verify(x[0:3].reset_index(drop=True))

Start loading model.joblib from model directory /home/datascience/RedBull-Racining-TimeToPit/md-rbc-Pipeline-model ...
Model is successfully loaded.


{'prediction': [4, 0, 0]}

The `.summary_status()` method is updated to show that the verify step has been completed.

In [32]:
sklearn_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Not Available Deployed the model                                                
predict() Not Available Called deployment predict endpoint

<a id='serialize_save'></a>
## Save

Once you are satisfied with the performance of the model and have verified that the `score.py` file is working, you can save the model to the model catalog. You do this with the `.save()` method on a `SklearnModel` object. This bundles up the model artifact that you have created, and push it to the model catalog. It returns the model OCID.

In [33]:
model_id = sklearn_model.save(display_name='rbr_deployed_Model_v2')

Start loading model.joblib from model directory /home/datascience/RedBull-Racining-TimeToPit/md-rbc-Pipeline-model ...
Model is successfully loaded.
['score.py', 'output_schema.json', 'runtime.yaml', 'score-Copy1.py', 'input_schema.json', 'model.joblib']


loop1:   0%|          | 0/5 [00:00<?, ?it/s]

artifact:/tmp/saved_model_589caec5-05c6-49be-9aa4-835c34fa6b88.zip


In [40]:
model_id

'ocid1.datasciencemodel.oc1.eu-frankfurt-1.amaaaaaaeras5xiathsdvcodjxioccsl3bmmou7j2tihpuvuc4mfrwolejca'

<a id='serialize_deploy'></a>
## Deploy

With the model in the model catalog, you can use the `.deploy()` method of an `SklearnModel` object to deploy the model. This method allows you to specify the attributes of the deployment such as the display name, description, instance type and count, the maximum bandwidth, and logging groups. The next cell deployd the model with the default settings, except for the custom display name. The `.deploy()` method returns a `ModelDeployment` object.

In [41]:
%%time
deploy = sklearn_model.deploy(display_name='rbr_deployed_Model_v1',)

loop1:   0%|          | 0/6 [00:00<?, ?it/s]

CPU times: user 734 ms, sys: 34.9 ms, total: 769 ms
Wall time: 12min 39s


After deployment, the `.summary_status()` method shows that the model is ACTIVE and the `predict()` method is available.

In [42]:
sklearn_model.summary_status()

Actions Needed
Step      Status    Details                                                           
initiate  Done      Initiated the model                                               
prepare() Done      Generated runtime.yaml                                            
                    Generated score.py                                                
                    Serialized model                                                  
                    Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done      Local tested .predict from score.py                               
save()    Done      Conducted Introspect Test                                         
                    Uploaded artifact to model catalog                                
deploy()  ACTIVE    Deployed the model                                                
predict() Available Called deployment predict endpoint

<a id='invoke_endpoint'></a>
# Invoking the Endpoint

#### Invoke deployed model
1. load data from storage though for this demo we are using in memory data here.
2. create JSON string
3. Hit the endpoint

In [43]:
import ads
import requests
import io
import oci
from oci.signer import Signer
ads.set_auth("resource_principal")
auth = oci.auth.signers.get_resource_principals_signer()
import pandas as pd
import numpy as np

## since data is already in the memory therefore we are not reloading it. We use X_test_xgbc transform a few rows to JSON and invoke the end point


### Invoke 

In [45]:
## select a few records and create JOSN string
## Invoke the endpoint


input_data = x[0:3].to_json(orient='records')
endpoint = 'https://modeldeployment.eu-frankfurt-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaaeras5xiajfocdhkqrlgypfij4ftru23itrsasw55jjtj2ex5fpma/predict'
body = input_data 

body = {'data': input_data, 'data_type':"pandas.core.frame.DataFrame"}

requests.post(endpoint, json=body, auth=auth).json()

{'prediction': [4, 0, 0]}

<a id='cleanup'></a>
# Clean Up

This notebook created a model deployment and a model. This section cleans up those resources. 

The model deployment must be deleted before the model can be deleted. You use the `.delete_deployment()` method on the `SklearnModel` object to do this.

## Delete a Model Deployment
The `odsc` SDK provides two methods for deleting a model deployment. The `ModelDeployer` class has a static method delete(). This method can also be accessed on a `ModelDeployer` object. For example, this notebook has a variable `deployer` which is a `ModelDeployer` object. A model deployment can be deleted with:

deployer.delete(model_deployment_id=deployment_id)
or equivalently,

ModelDeployer.delete(model_deployment_id=deployment_id)
If you have a `ModelDeployment` object, there is a `delete()` method that will delete the model that is associated with that object. The optional parameter `wait_for_completion` accepts a boolean and determines if the process is blocking or not.

The next cell will use a `ModelDeployment` object to delete the model deployment that was created in this notebook.

In [ ]:
sklearn_model.delete_deployment(wait_for_completion=True)

When a model deployment is deleted, it deletes the load balancer instances associated with it. If logging was configured, it does not delete that log group or loggers. In addition, the model in the model catalog is not deleted. In the next cell, these resources will be removed.

In [ ]:
# Delete the log group and logs
# logging_client = oci.logging.LoggingManagementClient(config=oci.config.from_file())
# logging_client.delete_log(log_group_ocid, access_log_ocid)
# logging_client.delete_log(log_group_ocid, predict_log_ocid)
# logging_client.delete_log_group(log_group_ocid)

# Delete the model
ModelCatalog().delete_model(model_id)